In [ ]:
print("""
@File         : ch10_restructuring_data_into_a_tidy_form.ipynb
@Author(s)    : Stephen CUI
@LastEditor(s): Stephen CUI
@CreatedTime  : 2024-07-25 15:11:41
@Email        : cuixuanstephen@gmail.com
@Description  : 将数据重组为整洁的形式
""")

In [ ]:
%cd ../

In [ ]:
import pandas as pd
import numpy as np

什么是整洁数据？Hadley 提出了三个确定数据集是否整洁的指导原则：

1. 每个变量形成一列
2. 每个观察值形成一行
3. 每种类型的观察单位形成一个表格

解决混乱数据的第一步是识别它，因为它存在，而且有无限的可能性。哈德利明确提到了五种最常见的混乱数据类型：
1. 列名是值，而不是变量名
2. 多个变量存储在列名中
3. 变量存储在行和列中
4. 多种类型的观测单元存储在同一张表中
5. 单个观察单元存储在多个表中

## 使用 `stack` 将变量值整理为列名

In [ ]:
state_fruit = pd.read_csv('data/state_fruit.csv', index_col=0)
state_fruit

这张表看起来并没有什么杂乱之处，信息也很容易理解。然而，根据整洁原则，它并不整洁。每个列名都是一个变量的值。事实上，DataFrame 中甚至没有一个变量名。

In [ ]:
state_fruit.stack()

In [ ]:
(
    state_fruit.stack()
    .reset_index()
)

In [ ]:
(
    state_fruit.stack()
    .reset_index()
    .rename(columns={'level_0': 'state', 'level_1': 'fruit', 0: 'weight'})
)

In [ ]:
(
    state_fruit.stack()
    .rename_axis(['state', 'fruit'])
)

In [ ]:
(
    state_fruit.stack()
    .rename_axis(['state', 'fruit'])
    .reset_index(name='weight')
)

`.stack` 默认情况下，它会获取（层次结构列中最内层的）列名并对其进行转置，这样它们就成为新的最内层索引级别。请注意，每个旧列名仍通过与每个州配对来标记其原始值。3 x 3 DataFrame 中有 9 个原始值，它们被转换为具有相同数量值的单个 Series。原始的第一行数据成为结果 Series 中的前三个值。

使用 `.stack` 的关键之一是将您不想转换的所有列放在索引中。

In [ ]:
state_fruit2 = pd.read_csv('data/state_fruit2.csv')
state_fruit2

In [ ]:
state_fruit2.stack()

In [ ]:
state_fruit2.set_index('State').stack()

要正确重塑这些数据，您需要先使用 `.set_index` 方法将所有未重塑的列放入索引中，然后使用 `.stack`。

使用 `melt` 将变量值整理为列名

DataFrame 有一个名为 `.melt` 的方法，它类似于上一节中描述的 `.stack` 方法，但灵活性更高。

In [ ]:
state_fruit2

使用 `.melt` 方法，将适当的列传递给 `id_vars` 和 `value_vars` 参数：

In [ ]:
state_fruit2.melt(id_vars=['State'], value_vars=['Apple', 'Orange', 'Banana'])

默认情况下，`.melt` 将转换后的列名称称为变量，将相应的值称为值。方便的是，`.melt` 有两个附加参数，`var_name` 和 `value_name`，可让您重命名这两列：

In [ ]:
state_fruit2.melt(id_vars=['State'],
                  value_vars=['Apple', 'Orange', 'Banana'],
                  var_name='fruit',
                  value_name='weight')

`.melt` 方法重塑您的 DataFrame。它最多需要五个参数，其中两个对于理解如何正确重塑数据至关重要：

- `id_vars` 是要保留为列而不是 reshape
- `value_vars` 是要重塑为单个列的列名列表

id_vars （即标识变量）保留在同一列中，但对传递给value_vars 的每个列重复。`.melt` 的一个重要方面是它会忽略索引中的值，它会默默地删除索引并将其替换为默认的RangeIndex。这意味着，如果您的索引中确实有您想要保留的值，则在使用 `melt` 之前，您需要先重置索引。

In [ ]:
state_fruit2.melt()

In [ ]:
state_fruit2.melt(id_vars='State')

同时 `stack` 多组变量

In [ ]:
movie = pd.read_csv('data/movie.csv')
actor = movie[['movie_title', 'actor_1_name', 'actor_2_name',
               'actor_3_name', 'actor_1_facebook_likes', 'actor_2_facebook_likes',
               'actor_3_facebook_likes']]
actor.head()

In [ ]:
def change_col_name(col_name):
    col_name = col_name.replace('_name', '')
    if 'facebook' in col_name:
        fb_idx = col_name.find('facebook')
        col_name = (col_name[:5] + col_name[fb_idx - 1:]
                    + col_name[5: fb_idx - 1])
    return col_name

In [ ]:
actor2 = actor.rename(columns=change_col_name)

In [ ]:
actor2

In [ ]:
stubs = ['actor', 'actor_facebook_likes']
actor2_tidy = pd.wide_to_long(
    actor2, stubnames=stubs, i=['movie_title'], j='actor_num',
    sep="_"
)
actor2_tidy.head()

`wide_to_long` 函数的工作方式相当特殊。其主要参数是 `stubnames`，即字符串列表。每个字符串代表一个列分组。所有以此字符串开头的列都将堆叠到单个列中。在此配方中，有两组列：actor 和 actor_facebook_likes。默认情况下，这些列组中的每一组都需要以数字结尾。随后将使用此数字来标记重塑后的数据。这些列组中的每一组都有一个下划线字符，将 `stubname` 与结尾数字分隔开。为此，您必须使用 `sep` 参数。

除此之外，`wide_to_long` 需要一个唯一的列，即参数 `i`，作为不会被堆叠的标识变量。还需要参数 `j`，它重命名从原始列名末尾剥离的标识数字。

In [ ]:
df = pd.read_csv('data/stackme.csv')
df

In [ ]:
df.rename(columns = {'a1':'group1_a1', 'b2':'group1_b2',
                     'd':'group2_a1', 'e':'group2_b2'})

In [ ]:
pd.wide_to_long(
    df.rename(columns = {'a1':'group1_a1', 'b2':'group1_b2',
                     'd':'group2_a1', 'e':'group2_b2'}),
    stubnames=['group1', 'group2'],
    i=['State', 'Country', 'Test'],
    j='Label', suffix='.+', sep='_'
)

## 反转堆叠数据

DataFrames 有两个类似的方法，`.stack` 和 `.melt`，用于将水平列名称转换为垂直列值。DataFrames 可以分别使用 `.unstack` 和 `.pivot` 方法反转这两个操作。`.stack` 和 `.unstack` 是仅允许控制列和行索引的方法，而 `.melt` 和 `.pivot` 可以更灵活地选择要重塑哪些列。

In [ ]:
def usecol_func(name):
    return 'UGDS_' in name or name == 'INSTNM'

college = pd.read_csv('data/college.csv',
                      index_col='INSTNM', usecols=usecol_func)
college

In [ ]:
college_stacked = college.stack()
college_stacked

In [ ]:
college_stacked.unstack()

In [ ]:
college2 = pd.read_csv('data/college.csv', usecols=usecol_func)
college2

In [ ]:
college_melted = college2.melt(id_vars='INSTNM', var_name='Race', 
                               value_name='Percentage')
college_melted

In [ ]:
college_melted.pivot(index='INSTNM', columns='Race', values='Percentage')

In [ ]:
college.stack().unstack()

In [ ]:
college.T

## 分组聚合后取消堆叠

按单个列对数据进行分组并对单个列执行聚合将返回易于使用的结果。按多个列进行分组时，生成的聚合可能不以易于使用的方式构建。由于 `.groupby` 操作默认将唯一的分组列放在索引中，因此 `.unstack` 方法可以有利于重新排列数据，以便以更有利于解释的方式呈现数据。

In [ ]:
employee = pd.read_csv('data/employee.csv')
(
    employee.groupby('RACE')['BASE_SALARY']
    .mean()
    .astype(int)
)

In [ ]:
(
    employee.groupby(['RACE', 'GENDER'])['BASE_SALARY']
    .mean()
    .astype(int)
)

In [ ]:
(
    employee.groupby(['RACE', 'GENDER'])['BASE_SALARY']
    .mean()
    .astype(int)
    .unstack()
)

In [ ]:
(
    employee.groupby(['RACE', 'GENDER'])['BASE_SALARY']
    .mean()
    .astype(int)
    .unstack('RACE')
)

In [ ]:
(
    employee.groupby(['RACE', 'GENDER'])['BASE_SALARY']
    .agg(['mean', 'max', 'min'])
    .astype(int)
)

In [ ]:
(
    employee.groupby(['RACE', 'GENDER'])['BASE_SALARY']
    .agg(['mean', 'max', 'min'])
    .astype(int)
    .unstack('GENDER')
)

In [ ]:
(
    employee.groupby(['RACE', 'GENDER'])['BASE_SALARY']
    .agg(['mean', 'max', 'min'])
    .astype(int)
    .unstack('RACE')
)